<a href="https://colab.research.google.com/github/svangenderen/Julia/blob/main/Python_vs_Julia_vs_Numba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.9.3" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Installing Julia 1.9.3 on the current Colab Runtime...
2023-11-30 01:01:46 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.9/julia-1.9.3-linux-x86_64.tar.gz [146268149/146268149] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package BenchmarkTools...
Installing IJulia kernel...
[ Info: Installing julia kernelspec in /root/.local/share/jupyter/kernels/julia-1.9

Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then
jump to the 'Checking the Installation' section.


In [2]:
# Julia
@time for i in 1:100
    a = rand(1000, 1000)
    b = rand(1000)
    x = a \ b
end

  3.420835 seconds (700 allocations: 1.492 GiB, 3.14% gc time)
  3.289791 seconds (700 allocations: 1.492 GiB, 1.71% gc time)


In [2]:
# Python with NumPy
import numpy as np
from time import time



start = time()
for i in range(100):
    a = np.random.rand(1000, 1000)
    b = np.random.rand(1000)
    x = np.linalg.solve(a, b)
end = time()

print(end - start)

7.275834083557129


In [3]:
# Python with Numba
from numba import jit
import numpy as np
from time import time

@jit(nopython=True)
def solve_equation(a, b):
    n = len(b)
    x = np.empty(n)

    for i in range(n):
        x[i] = b[i] / a[i, i]
        for j in range(i + 1, n):
            b[j] -= a[j, i] * x[i]

    for i in range(n - 1, -1, -1):
        for j in range(i + 1, n):
            x[i] -= a[i, j] * x[j]
        x[i] /= a[i, i]

    return x

@jit(nopython=True)
def run_code(x):
    for i in range(x):
        a = np.random.rand(1000, 1000)
        b = np.random.rand(1000)
        c = solve_equation(a, b)

start = time()
run_code(100)
end = time()
print((end - start), "seconds")

2.7883758544921875 seconds
